In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

In [ ]:
file_path = 'Match Information in CSV format extracted from "Extract Information.ipynb"'
match_details_df = pd.read_csv(file_path)
match_details_df.head()

In [ ]:
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['Match ID']
  ys = series['Keypoint ID (Moving)']

  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = match_details_df.sort_values('Match ID', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Match ID')
_ = plt.ylabel('Keypoint ID (Moving)')

In [ ]:
match_details_df['Magnitude'] = np.sqrt(match_details_df['Displacement Vector Δx']**2 +
                                        match_details_df['Displacement Vector Δy']**2)
match_details_df['Direction'] = np.degrees(np.arctan2(match_details_df['Displacement Vector Δy'],
                                                      match_details_df['Displacement Vector Δx']))
match_details_df[['Match ID', 'Displacement Vector Δx', 'Displacement Vector Δy', 'Magnitude', 'Direction']].head()

In [ ]:
plt.figure(figsize=(9, 7))

# Creating a quiver plot with refined styling
quiver = plt.quiver(
    np.zeros(len(match_details_df)),
    np.zeros(len(match_details_df)),
    match_details_df['Displacement Vector Δx'],
    match_details_df['Displacement Vector Δy'],
    match_details_df['Magnitude'],
    scale=1, scale_units='xy', angles='xy', cmap='magma', edgecolor='k', linewidth=0.2
)

# Adding a color bar with modern tick parameters and larger font size
cbar = plt.colorbar(quiver, label='Magnitude')
cbar.ax.tick_params(labelsize=12)

# Setting labels and title with larger fonts and modern font styles
plt.xlabel('Displacement Vector Δx', fontsize=12, color='black')
plt.ylabel('Displacement Vector Δy', fontsize=12, color='black')
plt.title('Spatial Distribution of Displacement Vectors', fontsize=12, color='black')

# Adding a grid with a modern appearance
plt.grid(True, linestyle='--', linewidth=0.4, color='grey', alpha=0.4)

# Setting axis to be equal and adding a light background for a polished look
plt.axis('equal')
plt.gca().set_facecolor('#F5F5F5')

# Adding axis spines and tick parameters for a modern look
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_color('black')
plt.gca().spines['bottom'].set_color('black')
plt.tick_params(axis='x', colors='black', direction='out', length=3, width=2)
plt.tick_params(axis='y', colors='black', direction='out', length=3, width=2)

# Show plot with tight layout for better spacing
plt.tight_layout()
plt.show()

In [ ]:
descriptive_stats = match_details_df['Magnitude'].describe()
correlation_matrix = match_details_df[['Magnitude', 'Scale Difference', 'Orientation Difference']].corr()
descriptive_stats
correlation_matrix

In [ ]:
fixed_image_file = 'Fixed Information in CSV format extracted from "Extract Information.ipynb"'
moving_image_file = 'Moving Information in CSV format extracted from "Extract Information.ipynb"'
fixed_image_df = pd.read_csv(fixed_image_file)
moving_image_df = pd.read_csv(moving_image_file)
fixed_image_df.head(), moving_image_df.head()

In [ ]:
merged_df = match_details_df.merge(fixed_image_df[['Keypoint ID', 'X Coordinate', 'Y Coordinate']],
                                   left_on='Keypoint ID (Fixed)', right_on='Keypoint ID', how='left')
merged_df.rename(columns={'X Coordinate': 'X Coordinate (Fixed)',
                          'Y Coordinate': 'Y Coordinate (Fixed)'}, inplace=True)
merged_df.drop('Keypoint ID', axis=1, inplace=True)

merged_df = merged_df.merge(moving_image_df[['Keypoint ID', 'X Coordinate', 'Y Coordinate']],
                            left_on='Keypoint ID (Moving)', right_on='Keypoint ID', how='left')
merged_df.rename(columns={'X Coordinate': 'X Coordinate (Moving)',
                          'Y Coordinate': 'Y Coordinate (Moving)'}, inplace=True)
merged_df.drop('Keypoint ID', axis=1, inplace=True)

merged_df.head()

In [ ]:
correlation_matrix_extended = merged_df[['Magnitude', 'Scale Difference', 'Orientation Difference',
                                         'X Coordinate (Fixed)', 'Y Coordinate (Fixed)',
                                         'X Coordinate (Moving)', 'Y Coordinate (Moving)']].corr()

correlation_matrix_extended

#### Vector Field Visualization of Displacement Vectors

In [ ]:
plt.figure(figsize=(9, 7))

# Creating a quiver plot with refined styling
quiver = plt.quiver(
    merged_df['X Coordinate (Moving)'],
    merged_df['Y Coordinate (Moving)'],
    merged_df['Displacement Vector Δx'],
    merged_df['Displacement Vector Δy'],
    merged_df['Magnitude'],
    scale=1, scale_units='xy', angles='xy', cmap='magma', width=0.005, edgecolor='k', linewidth=0.2
)

# Adding a color bar with modern tick parameters and larger font size
cbar = plt.colorbar(quiver, label='Magnitude')
cbar.ax.tick_params(labelsize=12)

# Setting labels and title with larger fonts and modern font styles
plt.xlabel('X Coordinate (Moving)', fontsize=12, color='black')
plt.ylabel('Y Coordinate (Moving)', fontsize=12, color='black')
plt.title('Vector Field Visualization of Displacement Vectors', fontsize=12, color='black')

# Adding a grid with a modern appearance
plt.grid(True, linestyle='--', linewidth=0.4, color='grey', alpha=0.4)

# Setting axis to be equal and adding a light background for a polished look
plt.axis('equal')
plt.gca().set_facecolor('#F5F5F5')

# Adding axis spines and tick parameters for a modern look
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_color('black')
plt.gca().spines['bottom'].set_color('black')
plt.tick_params(axis='x', colors='black', direction='out', length=3, width=2)
plt.tick_params(axis='y', colors='black', direction='out', length=3, width=2)

# Show plot with tight layout for better spacing
plt.tight_layout()
plt.show()=

#### Magnitude Heatmap

In [ ]:
def interpolate_to_grid(x, y, z, x_grid, y_grid):
    z_grid = griddata((x, y), z, (x_grid, y_grid), method='cubic')
    return z_grid

x_coords = np.linspace(merged_df['X Coordinate (Moving)'].min(), merged_df['X Coordinate (Moving)'].max(), 100)
y_coords = np.linspace(merged_df['Y Coordinate (Moving)'].min(), merged_df['Y Coordinate (Moving)'].max(), 100)
x_grid, y_grid = np.meshgrid(x_coords, y_coords)

magnitude_grid = interpolate_to_grid(merged_df['X Coordinate (Moving)'], merged_df['Y Coordinate (Moving)'],
                                     merged_df['Magnitude'], x_grid, y_grid)

# Plotting the Magnitude Heatmap
fig, ax = plt.subplots(figsize=(9, 7))
cmap = ax.imshow(magnitude_grid, extent=(x_coords.min(), x_coords.max(), y_coords.min(), y_coords.max()),
                 origin='lower', cmap='magma', aspect='auto')

cbar = fig.colorbar(cmap, ax=ax, label='Magnitude')
cbar.ax.tick_params(labelsize=12)

ax.set_title('Magnitude Heatmap', fontsize=12, color='black')
ax.set_xlabel('X Coordinate (Moving)', fontsize=12, color='black')
ax.set_ylabel('Y Coordinate (Moving)', fontsize=12, color='black')

# Adding a grid with a modern appearance
ax.grid(True, linestyle='--', linewidth=0.5, color='grey', alpha=0.4)

# Adding axis spines and tick parameters for a modern look
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.tick_params(axis='x', colors='black', direction='out', length=3, width=2)
ax.tick_params(axis='y', colors='black', direction='out', length=3, width=2)

plt.tight_layout()
plt.show()

#### Direction Heatmap

In [ ]:
def interpolate_to_grid(x, y, z, x_grid, y_grid):
    z_grid = griddata((x, y), z, (x_grid, y_grid), method='cubic')
    return z_grid

x_coords = np.linspace(merged_df['X Coordinate (Moving)'].min(), merged_df['X Coordinate (Moving)'].max(), 100)
y_coords = np.linspace(merged_df['Y Coordinate (Moving)'].min(), merged_df['Y Coordinate (Moving)'].max(), 100)
x_grid, y_grid = np.meshgrid(x_coords, y_coords)

direction_grid = interpolate_to_grid(merged_df['X Coordinate (Moving)'], merged_df['Y Coordinate (Moving)'],
                                     merged_df['Direction'], x_grid, y_grid)

# Plotting the Direction Heatmap
fig, ax = plt.subplots(figsize=(9, 7))
cmap = ax.imshow(direction_grid, extent=(x_coords.min(), x_coords.max(), y_coords.min(), y_coords.max()),
                 origin='lower', cmap='hsv', aspect='auto')

cbar = fig.colorbar(cmap, ax=ax, label='Direction (Degrees)')
cbar.ax.tick_params(labelsize=12)

ax.set_title('Direction Heatmap', fontsize=12, color='black')
ax.set_xlabel('X Coordinate (Moving)', fontsize=12, color='black')
ax.set_ylabel('Y Coordinate (Moving)', fontsize=12, color='black')

# Adding a grid with a modern appearance
ax.grid(True, linestyle='--', linewidth=0.5, color='grey', alpha=0.4)

# Adding axis spines and tick parameters for a modern look
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.tick_params(axis='x', colors='black', direction='out', length=3, width=2)
ax.tick_params(axis='y', colors='black', direction='out', length=3, width=2)

plt.tight_layout()
plt.show()

#### Create Final CSV

In [ ]:
columns_to_keep = [
    "X Coordinate (Moving)",
    "Y Coordinate (Moving)",
    "Displacement Vector Δx",
    "Displacement Vector Δy",
    "Magnitude",
    "Direction",
    "Distance/Similarity Score",
    "Scale Difference",
    "Orientation Difference"
]

filtered_df = merged_df[columns_to_keep]
filtered_df.to_csv("output.csv", index=False)

In [ ]:
final_csv_path = 'output.csv'
final_csv_df = pd.read_csv(final_csv_path)